In [8]:
!pip install accelerate -q

In [1]:
import pprint

import os
import pathlib
import glob

import json
import geojson
from typing import Any, Dict
from dotenv import load_dotenv

import torch
import transformers
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoConfig
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast

import langchain
import langchain_community
import langchain_core

In [2]:
load_dotenv('./config.env')

True

In [3]:
MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct' #"meta-llama/Meta-Llama-3-8B"
HF_TOKEN: str | None = os.environ.get('HF_TOKEN')
SAVE_FOLDER = pathlib.Path('./llama_answers_dataset').resolve()
SAVE_FOLDER.mkdir(exist_ok=True, parents=True)

In [4]:
def get_prompt(question: str, context: str, *args, **kwargs) -> str:
    """Function for intialization of LLAMA3 prompt template."""

    default: str = """Ты - умный ассистент. Разработанный для помощи пользователю в его запросах."""
    default_rules: str = """Ответь на вопрос, основываясь на представленном ниже контексте. 
    Если ты не знаешь ответ, просто скажи, что не знаешь. Для ответа используй максимум три предложения, 
    дай краткое описание по имеющеся информации. Максимальная длина ответа 300 символов. 
    Если в описании оснащения здания рядом с каким-то пунктом стоит 0, это значит, 
    что описанный объект либо отсутствует, либо не исправен. 
    В тексте 0 - означает нет, 1 - означает да (про аварийное состояние здания). 
    Если указано значение None, значит информация в базе данных отсутствует. 
    У ответа должны быть единицы измерения. Если в одном здании несколько организаций, укажи их все. 
    Адрес здания (улица, номер дома, корпус) в вопросе пользователя должен полностью совпадать 
    с данными из контекста. """
    system_prompt: str = kwargs.get('system_prompt', default)
    rules: str = kwargs.get('additional_rules', default_rules)
    template: str = f"""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            {system_prompt}<|eot_id|>
            <|start_header_id|>user<|end_header_id|>
            {rules} Контекст :{context} Вопрос: {question}<|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """
    return template

In [5]:
def save_answer_as_json(answer: Dict):
    total_files_in_folder = len(glob.glob1(SAVE_FOLDER, '*.json'))
    f_pref = 'llama_ans'
    with open((SAVE_FOLDER/f'{f_pref}_{total_files_in_folder + 1}.json').resolve(), 'w', encoding='utf-8') as pth:
        json.dump(answer, pth)
    

In [6]:
def get_query(idx: int):
    """Load set of queries and cintext to them."""
    with open(f'./data/datasets/data_{idx}.json') as json_data:
        questions = json.load(json_data)
    with open(f'./data/buildings/buildings_part_{idx}.geojson') as buildings_data:
        manual_context = geojson.load(buildings_data)
    return questions, manual_context

In [7]:
def multi_ans(model: Any, amount: int = 10, **kwargs):
    """
    Get multiple answers from given model.
    """
    generation_temperature = kwargs.get('temperature', .5)
    for i in range(amount):
        queries, context = get_query(i)
        total_questions = list(queries.keys())
        for q_id in range(len(total_questions) // 5):
            # Pick one query from the list
            question_response_pair = queries[total_questions[q_id]]            
            query, response = question_response_pair['query'], question_response_pair['response']

            #Form a prompt from query and context
            prompt = get_prompt(question=query, context=context)
            answer = model(prompt, temperature=generation_temperature)

            json_ans = {'query': query, 
                        'llama_answer': answer[0]["generated_text"].split("<|end_header_id|>")[-1], 
                        'ideal_ans': response}
            save_answer_as_json(json_ans)
    print(f'Answers have been saved to {SAVE_FOLDER}. Amount: {len(glob.glob1(SAVE_FOLDER, "*.json"))}')
            # pprint.pp(json_ans)
            # print(f'Q: {query}')
            # print(f'R: {answer}')
            # break
            
        # question_id = list(questions.keys())[-40]
        # question_response_pair = questions[question_id]
        # manual_question = question_response_pair['query']
        # target = question_response_pair['response']
        
        # print(f'Question: {manual_question}')
        # print(f'Desire response: {target}')

In [8]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True)
model_config = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True, max_new_tokens=12000)

/root/miniconda3/envs/llama3-urbanistic/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    #quantization_config=quantization_config,
    torch_dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)
model.eval()
print('Model is ready.')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model is ready.


In [10]:
tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

/root/miniconda3/envs/llama3-urbanistic/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
pipeline = pipeline(
    'text-generation', 
    model=model,
    tokenizer=tokenizer,
    max_length=12000,
    device_map='auto',
    )

In [12]:
## MULTI ans testing
multi_ans(pipeline, amount=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

Answers have been saved to /home/hdd/work/llms/BIAM-Urb/llama_answers_dataset. Amount: ['llama_ans_70.json', 'llama_ans_71.json', 'llama_ans_72.json', 'llama_ans_73.json', 'llama_ans_74.json', 'llama_ans_75.json', 'llama_ans_76.json', 'llama_ans_77.json', 'llama_ans_78.json', 'llama_ans_79.json', 'llama_ans_80.json', 'llama_ans_81.json', 'llama_ans_82.json', 'llama_ans_83.json', 'llama_ans_84.json', 'llama_ans_85.json', 'llama_ans_86.json', 'llama_ans_87.json', 'llama_ans_88.json', 'llama_ans_89.json', 'llama_ans_90.json', 'llama_ans_91.json', 'llama_ans_92.json', 'llama_ans_93.json', 'llama_ans_94.json', 'llama_ans_95.json', 'llama_ans_96.json', 'llama_ans_97.json', 'llama_ans_98.json', 'llama_ans_99.json', 'llama_ans_100.json', 'llama_ans_101.json', 'llama_ans_102.json', 'llama_ans_103.json', 'llama_ans_104.json', 'llama_ans_105.json', 'llama_ans_106.json', 'llama_ans_107.json', 'llama_ans_108.json', 'llama_ans_109.json', 'llama_ans_110.json', 'llama_ans_111.json', 'llama_ans_112.jso

In [10]:
id = 450

with open(f'./data/datasets/data_{id}.json') as json_data:
    questions = json.load(json_data)
with open(f'./data/buildings/buildings_part_{id}.geojson') as buildings_data:
    manual_context = geojson.load(buildings_data)
    

In [11]:
print(f'Questions:\n\n{questions}\n\n')
print(f'Context:\n\n{manual_context}\n\n')

Questions:

{'4500_0': {'building_id': 117031, 'query': 'Какой идентификатор физического объекта у дома по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А"?', 'response': 'Идентификатор дома по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А" – 117031.', 'target_value': 117031}, '4500_1': {'building_id': 117031, 'query': 'В каком районе находится дом по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А"?', 'response': 'Дом по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А" находится в районе "Курортный".', 'target_value': 'Курортный'}, '4500_2': {'building_id': 117031, 'query': 'В каком муниципальном образовании находится дом по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А"?', 'response': 'Дом по адресу "Санкт-Петербург, город Зеленогорск, Овражная улица, дом 29, литера А" находится в муниципальном образовании Зеленогорск.', 'target_value': 

In [12]:
# File in question has multiple varinats of questions, we have to pick one
question_id = list(questions.keys())[-40]
question_response_pair = questions[question_id]
manual_question = question_response_pair['query']
target = question_response_pair['response']

print(f'Question: {manual_question}')
print(f'Desire response: {target}')


Question: Какая площадь основания здания по адресу "Санкт-Петербург, Володарского, 60"?
Desire response: Площадь основания здания по адресу "Санкт-Петербург, Володарского, 60" равна 1352 кв. м.


In [11]:
# query_1 = """Ответь на вопрос, основываясь на представленном ниже контексте. Если ты не знаешь ответ, просто скажи, что не знаешь. Для ответа используй максимум три предложения, дай краткое описание по имеющеся информации. Максимальная длина ответа 300 символов. Если в описании оснащения здания рядом с каким-то пунктом стоит 0, это значит, что описанный объект либо отсутствует, либо не исправен. В тексте 0 - означает нет, 1 - означает да (про аварийное состояние здания). Если указано значение None, значит информация в базе данных отсутствует. У ответа должны быть единицы измерения. Если в одном здании несколько организаций, укажи их все. Адрес здания (улица, номер дома, корпус) в вопросе пользователя должен полностью совпадать с данными из контекста. 
#               Контекст :{"crs": {"properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}, "type": "name"}, "features": [{"geometry": {"coordinates": [30.413192, 59.931727], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Перевозный переулок, 19 к1", "administrative_unit": "Красногвардейский", "block_id": 966, "building_area": 544.9767456054688, "building_year": 1970.0, "capacity": 104, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112667, "lift_count": 2.0, "living_area": 4253.52978515625, "municipality": "Малая Охта", "name": "Жилой дом", "population_balanced": 182.0, "project_type": "1-528", "refusechute": "1", "repair_years": "2008; 2009; 2013", "resident_number": 137.0, "storeys_count": 12.0, "ukname": "ЖСК №351"}, "type": "Feature"}, {"geometry": {"coordinates": [30.446375, 60.002483], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Пискарёвский проспект, 143", "administrative_unit": "Красногвардейский", "block_id": 209, "building_area": 625.3032836914062, "building_year": 1963.0, "capacity": 979, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112668, "lift_count": null, "living_area": 1988.5999755859375, "municipality": "Полюстрово", "name": "Жилой дом", "population_balanced": 44.0, "project_type": "Типовой", "refusechute": "0", "repair_years": "2003; 2007", "resident_number": 127.0, "storeys_count": 4.0, "ukname": "ООО \"Жилкомсервис  №2 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.478519, 59.942259], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Индустриальный проспект, 15", "administrative_unit": "Красногвардейский", "block_id": 514, "building_area": 2208.093994140625, "building_year": 1987.0, "capacity": 38, "central_electricity": "1", "central_gas": "0", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112669, "lift_count": 10.0, "living_area": 15714.0, "municipality": "Пороховые", "name": "Жилой дом", "population_balanced": 786.0, "project_type": "137 инд. 1эт", "refusechute": "1", "repair_years": "2006; 2011", "resident_number": 890.0, "storeys_count": 12.0, "ukname": "ООО \"Жилкомсервис №2 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.474802, 59.950229], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Индустриальный проспект, 29", "administrative_unit": "Красногвардейский", "block_id": 1000, "building_area": 1654.4703369140625, "building_year": 1981.0, "capacity": 132, "central_electricity": "1", "central_gas": "0", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112670, "lift_count": 6.0, "living_area": 14881.900390625, "municipality": "Пороховые", "name": "Жилой дом", "population_balanced": 744.0, "project_type": "137", "refusechute": "1", "repair_years": "2007; 2008; 2009; 2010; 2012", "resident_number": 768.0, "storeys_count": 16.0, "ukname": "ООО \"Жилкомсервис  №2 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.412829, 59.957531], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Синявинская, 3", "administrative_unit": "Красногвардейский", "block_id": 1513, "building_area": 920.9356079101562, "building_year": 1955.0, "capacity": 4026, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112671, "lift_count": null, "living_area": 2921.7900390625, "municipality": "Большая Охта", "name": "Жилой дом", "population_balanced": 109.0, "project_type": "Индивидуальный", "refusechute": "0", "repair_years": "2005; 2008", "resident_number": 151.0, "storeys_count": 5.0, "ukname": "ООО \"Жилкомсервис  №1 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.412394, 59.930392], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Новочеркасский проспект, 24", "administrative_unit": "Красногвардейский", "block_id": 966, "building_area": 1029.1322021484375, "building_year": 1961.0, "capacity": 11541, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112672, "lift_count": null, "living_area": 3446.81005859375, "municipality": "Малая Охта", "name": "Жилой дом", "population_balanced": 147.0, "project_type": "1-335", "refusechute": "0", "repair_years": "2005; 2007; 2008", "resident_number": 157.0, "storeys_count": 5.0, "ukname": "ООО \"Жилкомсервис  №1 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.482305, 59.9406], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Ленская, 5", "administrative_unit": "Красногвардейский", "block_id": 584, "building_area": 842.4454345703125, "building_year": 1983.0, "capacity": 208, "central_electricity": "1", "central_gas": "0", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112673, "lift_count": 3.0, "living_area": 5957.89990234375, "municipality": "Пороховые", "name": "Жилой дом", "population_balanced": 298.0, "project_type": "9664", "refusechute": "1", "repair_years": "2019", "resident_number": 316.0, "storeys_count": 15.0, "ukname": "ООО \"Жилкомсервис №2  Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.413381, 59.951406], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Малыгина, 6", "administrative_unit": "Красногвардейский", "block_id": 1245, "building_area": 608.9186401367188, "building_year": 1902.0, "capacity": 191, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112674, "lift_count": null, "living_area": 1585.199951171875, "municipality": "Большая Охта", "name": "Жилой дом", "population_balanced": 59.0, "project_type": "Индивидуальный", "refusechute": "0", "repair_years": "2004; 2005; 2011; 2012", "resident_number": 96.0, "storeys_count": 4.0, "ukname": "ООО \"Жилкомсервис № 1 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.497059, 59.952257], "type": "Point"}, "properties": {"address": "Санкт-Петербург, проспект Энтузиастов, 53, Коммуны, 38", "administrative_unit": "Красногвардейский", "block_id": 454, "building_area": 4079.040283203125, "building_year": 1982.0, "capacity": 205, "central_electricity": "1", "central_gas": "0", "central_heating": "1", "central_hot_water": "1", "central_water": "1", "failure": "0", "id": 112675, "lift_count": 15.0, "living_area": 28484.900390625, "municipality": "Ржевка", "name": "Жилой дом", "population_balanced": 1369.0, "project_type": "1ЛГ-504Д", "refusechute": "1", "repair_years": "2002; 2007; 2008; 2018", "resident_number": 1721.0, "storeys_count": 9.0, "ukname": "ООО \"Жилкомсервис  №2 Красногвардейского района\""}, "type": "Feature"}, {"geometry": {"coordinates": [30.489192, 59.959992], "type": "Point"}, "properties": {"address": "Санкт-Петербург, Коммуны, 54 к2", "administrative_unit": "Красногвардейский", "block_id": 404, "building_area": 569.5244750976562, "building_year": 1950.0, "capacity": 4884, "central_electricity": "1", "central_gas": "1", "central_heating": "1", "central_hot_water": "0", "central_water": "1", "failure": "0", "id": 112677, "lift_count": null, "living_area": 689.7899780273438, "municipality": "Ржевка", "name": "Жилой дом", "population_balanced": 33.0, "project_type": "Индивидуальный", "refusechute": "0", "repair_years": "2004; 2011", "resident_number": 41.0, "storeys_count": 2.0, "ukname": "ООО \"Жилкомсервис №2 Красногвардейского района\""}, "type": "Feature"}], "type": "FeatureCollection"}
#               Вопрос: Есть ли в доме по адресу "Санкт-Петербург, Перевозный переулок, 19 к1" централизованная электрификация (электричество)?"""
# answer_1 = pipeline(query_1)


In [12]:
# print(answer_1[0]['generated_text'])

In [14]:
query: str = get_prompt(question=manual_question, context=manual_context)
# print(query)

In [15]:
answer: Any = pipeline(query, temperature=0.2)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [16]:
print(f'Вопрос: {manual_question}')
print(f'Верный ответ: {target}')
print(f'Ответ: {answer[0]["generated_text"].split("<|end_header_id|>")[-1]}')

Вопрос: Какая площадь основания здания по адресу "Санкт-Петербург, Володарского, 60"?
Верный ответ: Площадь основания здания по адресу "Санкт-Петербург, Володарского, 60" равна 1352 кв. м.
Ответ: 
             По адресу "Санкт-Петербург, Володарского, 60" здание с площадью основания 1351.758544921875 квадратных метров.


In [18]:
json_ans = {'query': manual_question, 
            'llama_answer': answer[0]["generated_text"].split("<|end_header_id|>")[-1], 
            'ideal_ans': target}

In [27]:
save_path = pathlib.Path('./llama_answers_dataset')
save_path.mkdir(exist_ok=True, parents=True)
f_pref = 'llama_ans'
idx = 0

with open((save_path/f'{f_pref}_{idx}.json').resolve(), 'w') as pth:
    json.dump(json_ans, pth)


In [25]:
(save_path/f'{f_pref}_{idx}.json').resolve()

PosixPath('/home/hdd/work/llms/BIAM-Urb/llama_answers_dataset/llama_ans_0.json')